In [1]:
)clear
⎕IO←0

clear ws

# Grain growth model

## Introduction

This document describes a grain growth model. The model is similar to the one presented in [[Traka 2022]](https://doi.org/10.4233/uuid:962f6655-a1b8-4c38-8467-0b2b651ab629), but with some simplifications and written in [APL](https://dyalog.com).

## Model overview

Our goal is to simulate the evolution of a steel microstructure during a grain growth process. The microstructure will be represented as a 2D grid of square cells, where each cell has assigned a crystallographic orientation.

The model considers, for each cell, a transformation ratio corresponding to each of its first-order neighbours. During a simulation step, the ratio of transformation into each neighbour is calculated for all cells, and the time increment is selected such that at least one cell will transform (will reach a transformation ratio of 1).

The results of the model will be the final microstructure after the specified time and the evolution of grain size during the simulation.

## Growth rate of each neighbour

The calculation of the transformation rate can be performed taking into account that the energy difference resulting from the transformation must be equal to the work performed by the pressure at the boundary (or boundaries). A larger energy difference will imply a larger pressure and therefore a faster transformation rate.

### Simplified example

This section describes the procedure used for the calculation of transformation rates in APL. Instead of showing a real example with calculations, we will demonstrate how the model works using a simplified example.

#### Initial microstructure

Our sample microstructure is composed of 9 cells. We assume it is periodic. In the model, each cell will have associated a crystallographic orientation. In this example, we will just give them names corresponding to the cardinal (`N S W E`) and ordinal (`NW NE SW SE`) directions and the additional name `O` for the center cell.

We give to our microstructure the name `o`.

In [2]:
⊢o←3 3⍴'NW' 'N' 'NE' 'W' 'O' 'E' 'SW' 'S' 'SE'

┌──┬─┬──┐
│NW│N│NE│
├──┼─┼──┤
│W │O│E │
├──┼─┼──┤
│SW│S│SE│
└──┴─┴──┘

#### First-order neighbours

Now, we want to find the first-order neighbours. We will need to know which are the neighbours to calculate misorientations later.

Instead of searching neighbours for each of the individual cells one by one, what we want is a set of four arrays, each of them similar to the one that represents the initial microstructure (the array `o`), but corresponding to the neighbours for each cell in the four directions. We can obtain these "neighbour arrays" just rotating the original array along the vertical and horizontal directions.

In [3]:
⊢(n s w e)←¯1 1(⊖¨,⌽¨)⊂o  ⍝ north, sout, west and east neighbours

┌─────────┬─────────┬─────────┬─────────┐
│┌──┬─┬──┐│┌──┬─┬──┐│┌──┬──┬─┐│┌─┬──┬──┐│
││SW│S│SE│││W │O│E │││NE│NW│N│││N│NE│NW││
│├──┼─┼──┤│├──┼─┼──┤│├──┼──┼─┤│├─┼──┼──┤│
││NW│N│NE│││SW│S│SE│││E │W │O│││O│E │W ││
│├──┼─┼──┤│├──┼─┼──┤│├──┼──┼─┤│├─┼──┼──┤│
││W │O│E │││NW│N│NE│││SE│SW│S│││S│SE│SW││
│└──┴─┴──┘│└──┴─┴──┘│└──┴──┴─┘│└─┴──┴──┘│
└─────────┴─────────┴─────────┴─────────┘

Notice that, for the center cell, the neighbours in each direction will be `N`, `S`, `W` and `E`, as expected. For convenience, we will define a function `O` to extract the value of the center cell:

In [4]:
O←(⊂1 1)∘⊃
O¨n s w e

NSWE

#### Misorientation angle

At continuation, we will calculate the misorientation angle (or disorientation) between each cell and its neighbours. In the model, the disorientation function `D` will return an angle or some other representation of the misorientation between both crystallographic orientations. In our example, we will simply add a hyphen between the orientations (eg. the disorientation between the center cell `O` and the south cell `S` is `O-S`).

In [5]:
D←{⍺,'-',⍵}¨
2 2⍴(on os ow oe)←o∘D¨n s w e  ⍝ disorientation with north, south, west and east neighbour

┌─────────────────┬─────────────────┐
│┌─────┬───┬─────┐│┌─────┬───┬─────┐│
││NW-SW│N-S│NE-SE│││NW-W │N-O│NE-E ││
│├─────┼───┼─────┤│├─────┼───┼─────┤│
││W-NW │O-N│E-NE │││W-SW │O-S│E-SE ││
│├─────┼───┼─────┤│├─────┼───┼─────┤│
││SW-W │S-O│SE-E │││SW-NW│S-N│SE-NE││
│└─────┴───┴─────┘│└─────┴───┴─────┘│
├─────────────────┼─────────────────┤
│┌─────┬────┬────┐│┌────┬────┬─────┐│
││NW-NE│N-NW│NE-N│││NW-N│N-NE│NE-NW││
│├─────┼────┼────┤│├────┼────┼─────┤│
││W-E │O-W │E-O │││W-O │O-E │E-W ││
│├─────┼────┼────┤│├────┼────┼─────┤│
││SW-SE│S-SW│SE-S│││SW-S│S-SE│SE-SW││
│└─────┴────┴────┘│└────┴────┴─────┘│
└─────────────────┴─────────────────┘

#### Grain boundary energies and mobilities at each face

We use these values to calculate the corresponding boundary energies, applying the [Read-Shockley equation](#Boundary-energy). In this example, we will simply represent the boundary energy for the misorientation `O-N` as `G(O-N)`.

In [6]:
G←{'G(',⍵,')'}¨
2 2⍴(gon gos gow goe)←G¨on os ow oe  ⍝ boundary energy at north, south, west and east face

┌──────────────────────────┬──────────────────────────┐
│┌────────┬──────┬────────┐│┌────────┬──────┬────────┐│
││G(NW-SW)│G(N-S)│G(NE-SE)│││G(NW-W) │G(N-O)│G(NE-E) ││
│├────────┼──────┼────────┤│├────────┼──────┼────────┤│
││G(W-NW) │G(O-N)│G(E-NE) │││G(W-SW) │G(O-S)│G(E-SE) ││
│├────────┼──────┼────────┤│├────────┼──────┼────────┤│
││G(SW-W) │G(S-O)│G(SE-E) │││G(SW-NW)│G(S-N)│G(SE-NE)││
│└────────┴──────┴────────┘│└────────┴──────┴────────┘│
├──────────────────────────┼──────────────────────────┤
│┌────────┬───────┬───────┐│┌───────┬───────┬────────┐│
││G(NW-NE)│G(N-NW)│G(NE-N)│││G(NW-N)│G(N-NE)│G(NE-NW)││
│├────────┼───────┼───────┤│├───────┼───────┼────────┤│
││G(W-E) │G(O-W) │G(E-O) │││G(W-O) │G(O-E) │G(E-W) ││
│├────────┼───────┼───────┤│├───────┼───────┼────────┤│
││G(SW-SE)│G(S-SW)│G(SE-S)│││G(SW-S)│G(S-SE)│G(SE-SW)││
│└────────┴───────┴───────┘│└───────┴───────┴────────┘│
└──────────────────────────┴──────────────────────────┘

Boundary mobilities (`mon mos mow moe`) are calculated as a function of the disorientations and also the temperature `T`, using [Humphrey's law](#Boundary-mobility).

In [7]:
M←{'M(T,',⍵,')'}¨
2 2⍴(mon mos mow moe)←M¨on os ow oe  ⍝ boundary mobility at north, south, west and east face

┌────────────────────────────────┬────────────────────────────────┐
│┌──────────┬────────┬──────────┐│┌──────────┬────────┬──────────┐│
││M(T,NW-SW)│M(T,N-S)│M(T,NE-SE)│││M(T,NW-W) │M(T,N-O)│M(T,NE-E) ││
│├──────────┼────────┼──────────┤│├──────────┼────────┼──────────┤│
││M(T,W-NW) │M(T,O-N)│M(T,E-NE) │││M(T,W-SW) │M(T,O-S)│M(T,E-SE) ││
│├──────────┼────────┼──────────┤│├──────────┼────────┼──────────┤│
││M(T,SW-W) │M(T,S-O)│M(T,SE-E) │││M(T,SW-NW)│M(T,S-N)│M(T,SE-NE)││
│└──────────┴────────┴──────────┘│└──────────┴────────┴──────────┘│
├────────────────────────────────┼────────────────────────────────┤
│┌──────────┬─────────┬─────────┐│┌─────────┬─────────┬──────────┐│
││M(T,NW-NE)│M(T,N-NW)│M(T,NE-N)│││M(T,NW-N)│M(T,N-NE)│M(T,NE-NW)││
│├──────────┼─────────┼─────────┤│├─────────┼─────────┼──────────┤│
││M(T,W-E) │M(T,O-W) │M(T,E-O) │││M(T,W-O) │M(T,O-E) │M(T,E-W) ││
│├──────────┼─────────┼─────────┤│├─────────┼─────────┼──────────┤│
││M(T,SW-SE)│M(T,S-SW)│M(T,SE-S)│││M(T,SW-S)│M(T,S-SE)│M(T,SE-SW)││
│└──────────┴─────────┴─────────┘│└─────────┴─────────┴──────────┘│
└────────────────────────────────┴────────────────────────────────┘

#### Grain boundary energy of each cell

The boundary energies of the first-order neighbours can be added to obtain the current boundary energy of each cell, which we call `go`.

In [8]:
SUM←{⊃(⊣,¨'+',¨⊢)/⍵}
⊢go←SUM gon gos gow goe  ⍝ current boundary energy

┌─────────────────────────────────┬─────────────────────────────┬─────────────────────────────────┐
│G(NW-SW)+G(NW-W)+G(NW-NE)+G(NW-N)│G(N-S)+G(N-O)+G(N-NW)+G(N-NE)│G(NE-SE)+G(NE-E)+G(NE-N)+G(NE-NW)│
├─────────────────────────────────┼─────────────────────────────┼─────────────────────────────────┤
│G(W-NW)+G(W-SW)+G(W-E)+G(W-O) │G(O-N)+G(O-S)+G(O-W)+G(O-E) │G(E-NE)+G(E-SE)+G(E-O)+G(E-W) │
├─────────────────────────────────┼─────────────────────────────┼─────────────────────────────────┤
│G(SW-W)+G(SW-NW)+G(SW-SE)+G(SW-S)│G(S-O)+G(S-N)+G(S-SW)+G(S-SE)│G(SE-E)+G(SE-NE)+G(SE-S)+G(SE-SW)│
└─────────────────────────────────┴─────────────────────────────┴─────────────────────────────────┘

The boundary energy of the central cell is:

In [9]:
O go

G(O-N)+G(O-S)+G(O-W)+G(O-E)

#### Boundary energy after transformation

When a cell transforms, its boundary energy will change. If we assume that the environment of the cell remains unchanged, we will be able to calculate the new boundary energy at each face from the relative disorientations between the neighbours.

For instance, to find the energy corresponding to the transformation into the north neighbour, we calculate the boundary energy substituting the cell `O` by the cell `N`. So, we will need to know the disorientation between the north and the south, west and east neighbours (of course, the disorientation of an orientation with itself is always zero, so the `N-N` misorientation does not need to be calculated).

In [10]:
⊢(ns nw ne)←n∘D¨s w e  ⍝ disorientations with north neighbour
⊢(sn sw se)←s∘D¨n w e  ⍝ disorientations with south neighbour
⊢(wn ws we)←w∘D¨n s e  ⍝ disorientations with west neighbour
⊢(en es ew)←e∘D¨n s w  ⍝ disorientations with east neighbour

┌─────────────────┬─────────────────┬─────────────────┐
│┌─────┬───┬─────┐│┌─────┬────┬────┐│┌────┬────┬─────┐│
││SW-W │S-O│SE-E │││SW-NE│S-NW│SE-N│││SW-N│S-NE│SE-NW││
│├─────┼───┼─────┤│├─────┼────┼────┤│├────┼────┼─────┤│
││NW-SW│N-S│NE-SE│││NW-E │N-W │NE-O│││NW-O│N-E │NE-W ││
│├─────┼───┼─────┤│├─────┼────┼────┤│├────┼────┼─────┤│
││W-NW │O-N│E-NE │││W-SE │O-SW│E-S │││W-S │O-SE│E-SW ││
│└─────┴───┴─────┘│└─────┴────┴────┘│└────┴────┴─────┘│
└─────────────────┴─────────────────┴─────────────────┘

┌─────────────────┬─────────────────┬─────────────────┐
│┌─────┬───┬─────┐│┌─────┬────┬────┐│┌────┬────┬─────┐│
││W-SW │O-S│E-SE │││W-NE │O-NW│E-N │││W-N │O-NE│E-NW ││
│├─────┼───┼─────┤│├─────┼────┼────┤│├────┼────┼─────┤│
││SW-NW│S-N│SE-NE│││SW-E │S-W │SE-O│││SW-O│S-E │SE-W ││
│├─────┼───┼─────┤│├─────┼────┼────┤│├────┼────┼─────┤│
││NW-W │N-O│NE-E │││NW-SE│N-SW│NE-S│││NW-S│N-SE│NE-SW││
│└─────┴───┴─────┘│└─────┴────┴────┘│└────┴────┴─────┘│
└─────────────────┴─────────────────┴─────────────────┘

┌─────────────────┬─────────────────┬─────────────────┐
│┌─────┬────┬────┐│┌─────┬────┬────┐│┌────┬─────┬────┐│
││NE-SW│NW-S│N-SE│││NE-W │NW-O│N-E │││NE-N│NW-NE│N-NW││
│├─────┼────┼────┤│├─────┼────┼────┤│├────┼─────┼────┤│
││E-NW │W-N │O-NE│││E-SW │W-S │O-SE│││E-O │W-E │O-W ││
│├─────┼────┼────┤│├─────┼────┼────┤│├────┼─────┼────┤│
││SE-W │SW-O│S-E │││SE-NW│SW-N│S-NE│││SE-S│SW-SE│S-SW││
│└─────┴────┴────┘│└─────┴────┴────┘│└────┴─────┴────┘│
└─────────────────┴─────────────────┴─────────────────┘

┌─────────────────┬─────────────────┬─────────────────┐
│┌────┬────┬─────┐│┌────┬────┬─────┐│┌────┬─────┬────┐│
││N-SW│NE-S│NW-SE│││N-W │NE-O│NW-E │││N-NE│NE-NW│NW-N││
│├────┼────┼─────┤│├────┼────┼─────┤│├────┼─────┼────┤│
││O-NW│E-N │W-NE │││O-SW│E-S │W-SE │││O-E │E-W │W-O ││
│├────┼────┼─────┤│├────┼────┼─────┤│├────┼─────┼────┤│
││S-W │SE-O│SW-E │││S-NW│SE-N│SW-NE│││S-SE│SE-SW│SW-S││
│└────┴────┴─────┘│└────┴────┴─────┘│└────┴─────┴────┘│
└─────────────────┴─────────────────┴─────────────────┘

The boundary energy after the transformation into each of the neighbour cells is obtained adding all the corresponding energies:

In [11]:
⊢gn←SUM G¨ns nw ne  ⍝ boundary energy after transforming into north neighbour
⊢gs←SUM G¨sn sw se  ⍝ boundary energy after transforming into north neighbour
⊢gw←SUM G¨wn ws we  ⍝ boundary energy after transforming into west neighbour
⊢ge←SUM G¨en es ew  ⍝ boundary energy after transforming into east neighbour

┌────────────────────────┬──────────────────────┬────────────────────────┐
│G(SW-W)+G(SW-NE)+G(SW-N)│G(S-O)+G(S-NW)+G(S-NE)│G(SE-E)+G(SE-N)+G(SE-NW)│
├────────────────────────┼──────────────────────┼────────────────────────┤
│G(NW-SW)+G(NW-E)+G(NW-O)│G(N-S)+G(N-W)+G(N-E) │G(NE-SE)+G(NE-O)+G(NE-W)│
├────────────────────────┼──────────────────────┼────────────────────────┤
│G(W-NW)+G(W-SE)+G(W-S) │G(O-N)+G(O-SW)+G(O-SE)│G(E-NE)+G(E-S)+G(E-SW) │
└────────────────────────┴──────────────────────┴────────────────────────┘

┌────────────────────────┬──────────────────────┬────────────────────────┐
│G(W-SW)+G(W-NE)+G(W-N) │G(O-S)+G(O-NW)+G(O-NE)│G(E-SE)+G(E-N)+G(E-NW) │
├────────────────────────┼──────────────────────┼────────────────────────┤
│G(SW-NW)+G(SW-E)+G(SW-O)│G(S-N)+G(S-W)+G(S-E) │G(SE-NE)+G(SE-O)+G(SE-W)│
├────────────────────────┼──────────────────────┼────────────────────────┤
│G(NW-W)+G(NW-SE)+G(NW-S)│G(N-O)+G(N-SW)+G(N-SE)│G(NE-E)+G(NE-S)+G(NE-SW)│
└────────────────────────┴──────────────────────┴────────────────────────┘

┌────────────────────────┬────────────────────────┬──────────────────────┐
│G(NE-SW)+G(NE-W)+G(NE-N)│G(NW-S)+G(NW-O)+G(NW-NE)│G(N-SE)+G(N-E)+G(N-NW)│
├────────────────────────┼────────────────────────┼──────────────────────┤
│G(E-NW)+G(E-SW)+G(E-O) │G(W-N)+G(W-S)+G(W-E) │G(O-NE)+G(O-SE)+G(O-W)│
├────────────────────────┼────────────────────────┼──────────────────────┤
│G(SE-W)+G(SE-NW)+G(SE-S)│G(SW-O)+G(SW-N)+G(SW-SE)│G(S-E)+G(S-NE)+G(S-SW)│
└────────────────────────┴────────────────────────┴──────────────────────┘

┌──────────────────────┬────────────────────────┬────────────────────────┐
│G(N-SW)+G(N-W)+G(N-NE)│G(NE-S)+G(NE-O)+G(NE-NW)│G(NW-SE)+G(NW-E)+G(NW-N)│
├──────────────────────┼────────────────────────┼────────────────────────┤
│G(O-NW)+G(O-SW)+G(O-E)│G(E-N)+G(E-S)+G(E-W) │G(W-NE)+G(W-SE)+G(W-O) │
├──────────────────────┼────────────────────────┼────────────────────────┤
│G(S-W)+G(S-NW)+G(S-SE)│G(SE-O)+G(SE-N)+G(SE-SW)│G(SW-E)+G(SW-NE)+G(SW-S)│
└──────────────────────┴────────────────────────┴────────────────────────┘

There will be many situations in which the assumption of a fixed environment could be too strong. It may happen that, when a cell transforms, some of its neighbours will transform too. Most of these situations are difficult or even impossible to predict. However, in the case of having a contiguous boundary along several cells, in which all the boundary cells have exactly the same environment, we can safely assume that the whole boundary will move at the same time.

What we need to do is to identify the boundaries of neighbour cells. When a cell is transforming and there is an equal boundary next to it, we can assume that the neighbour cell will transform too. For this, we will need to know the disorientations between first and second order neighbours. Let's see an example.

If we have the microstructure `o`, and we are calculating the new boundary energy when transforming `O` into the north cell `N`, we will have to check if the boundaries between `W` and `NW`, or between `E` and `NE`, are grain boundaries equal to the one between `O` and `N`. The condition for the boundary between `W` and `NW`, for example, is that the cell `NW` belongs to the same grain as `N` and that `W` belongs to the same grain as `O`.

The first step is to identify the grain boundaries in our microstructure. We will define the function `L` and the variables `lon`, `los`, `low` and `loe` to know which disorientations are lower than the low angle boundary, which we call `l`.

In [12]:
L←{'l>',⍵}¨
⊢(lon los low loe)←L¨on os ow oe  ⍝ low disorientation angle at north, south, west and east

┌───────────────────────┬───────────────────────┬───────────────────────┬───────────────────────┐
│┌───────┬─────┬───────┐│┌───────┬─────┬───────┐│┌───────┬──────┬──────┐│┌──────┬──────┬───────┐│
││l>NW-SW│l>N-S│l>NE-SE│││l>NW-W │l>N-O│l>NE-E │││l>NW-NE│l>N-NW│l>NE-N│││l>NW-N│l>N-NE│l>NE-NW││
│├───────┼─────┼───────┤│├───────┼─────┼───────┤│├───────┼──────┼──────┤│├──────┼──────┼───────┤│
││l>W-NW │l>O-N│l>E-NE │││l>W-SW │l>O-S│l>E-SE │││l>W-E │l>O-W │l>E-O │││l>W-O │l>O-E │l>E-W ││
│├───────┼─────┼───────┤│├───────┼─────┼───────┤│├───────┼──────┼──────┤│├──────┼──────┼───────┤│
││l>SW-W │l>S-O│l>SE-E │││l>SW-NW│l>S-N│l>SE-NE│││l>SW-SE│l>S-SW│l>SE-S│││l>SW-S│l>S-SE│l>SE-SW││
│└───────┴─────┴───────┘│└───────┴─────┴───────┘│└───────┴──────┴──────┘│└──────┴──────┴───────┘│
└───────────────────────┴───────────────────────┴───────────────────────┴───────────────────────┘

For example, if `l>O-W` and `l>N-NW`, that will mean that when we transform `O` into `N`, `W` will transform into `NW`, and therfore the boundary energy at the west face after transformation should be zero. So, our energies after transformation (for the center cell) will be:

In [13]:
NAND←{'((',⍺,')⍲(',⍵,'))'}¨ ⋄ MUL←{⍺,'×',⍵}¨
O gn←SUM(G ns)((⊃0 ¯1 NAND.⊖⊂low)MUL G nw)((⊃0 ¯1 NAND.⊖⊂loe)MUL G ne)  ⍝ north transformation energy
O gs←SUM(G sn)((⊃0  1 NAND.⊖⊂low)MUL G sw)((⊃0  1 NAND.⊖⊂loe)MUL G se)  ⍝ north transformation energy
O gw←SUM(G we)((⊃0 ¯1 NAND.⊖⊂lon)MUL G wn)((⊃0 ¯1 NAND.⊖⊂los)MUL G ws)  ⍝ west transformation energy
O ge←SUM(G ew)((⊃0 ¯1 NAND.⊖⊂lon)MUL G en)((⊃0 ¯1 NAND.⊖⊂los)MUL G es)  ⍝ east transformation energy

G(N-S)+((l>O-W)⍲(l>N-NW))×G(N-W)+((l>O-E)⍲(l>N-NE))×G(N-E)

G(S-N)+((l>O-W)⍲(l>S-SW))×G(S-W)+((l>O-E)⍲(l>S-SE))×G(S-E)

G(W-E)+((l>O-N)⍲(l>N-S))×G(W-N)+((l>O-S)⍲(l>N-O))×G(W-S)

G(E-W)+((l>O-N)⍲(l>N-S))×G(E-N)+((l>O-S)⍲(l>N-O))×G(E-S)

#### Energy difference

The decrease of energy resulting of each transformation is obtained substracting the new energy from the current one:

In [14]:
SUB←{⍺,' - (',⍵,')'}¨
O ∆gn←go SUB gn  ⍝ increment of energy when transforming into north neighbour
O ∆gs←go SUB gs  ⍝ increment of energy when transforming into south neighbour
O ∆gw←go SUB gw  ⍝ increment of energy when transforming into west neighbour
O ∆ge←go SUB ge  ⍝ increment of energy when transforming into east neighbour

G(O-N)+G(O-S)+G(O-W)+G(O-E) - (G(N-S)+((l>O-W)⍲(l>N-NW))×G(N-W)+((l>O-E)⍲(l>N-NE))×G(N-E))

G(O-N)+G(O-S)+G(O-W)+G(O-E) - (G(S-N)+((l>O-W)⍲(l>S-SW))×G(S-W)+((l>O-E)⍲(l>S-SE))×G(S-E))

G(O-N)+G(O-S)+G(O-W)+G(O-E) - (G(W-E)+((l>O-N)⍲(l>N-S))×G(W-N)+((l>O-S)⍲(l>N-O))×G(W-S))

G(O-N)+G(O-S)+G(O-W)+G(O-E) - (G(E-W)+((l>O-N)⍲(l>N-S))×G(E-N)+((l>O-S)⍲(l>N-O))×G(E-S))

#### Force magnitude

From this energy difference, we can find the pressure that will be exerted on the face dividing by the cell volume. The force can be obtained multiplying this pressure by the area. In case only one neighbour cell is transforming, the value of the force is simply obtained multiplying the pressure by the area of a cell face. However, when more than one cell transforms, which will be the case whenever other neighbour cells share orientation with the parent cell, we will have to take into account the force at all transforming faces.

In 2D, the number of possible cases is small. For example, let's assume cell `N` is the parent cell into which the cell `O` will transform. These are the possibilities:

1. All other neighbours are different to `N`. Pressure only on `N` face
2. Neighbours `N` and `E` or `W` are the same. Pressure on two perpendicular faces
3. Neighbours `N` and `S` are the same. Pressure on two opposite faces
4. Neighbours `N` and `S` and `W` or `E` are the same. Pressure on two opposite faces and a perpendicular face
5. Neighbours `N`, `W` and `E` are the same. Pressure on two opposite faces and a perpendicular face
6. All neighbours are the same. Pressure on the four faces

The force at each face will be the result of multiplying the pressure by the face area. The total force will be obtained adding the contribution of all the faces.

In [15]:
⊢k1←(÷2)*⍨+.×⍨1
⊢k2←(÷2)*⍨+.×⍨1 1
⊢k3←(÷2)*⍨+.×⍨2 
⊢k4←(÷2)*⍨+.×⍨2 1
⊢k5←(÷2)*⍨+.×⍨1 2
⊢k6←(÷2)*⍨+.×⍨2 2

1

1.414213562

2

2.236067977

2.236067977

2.828427125

These values can be represented in a table where the rows and column indicate the number of transforming faces in horizontal and vertical directions.

In [16]:
⊢k←(÷2)*⍨∘.+⍨2*⍨⍳3

0 1 2
1 1.414213562 2.236067977
2 2.236067977 2.828427125

And we can also define a `K` function to reach values from this table:

In [17]:
K←(,k)⌷⍨∘⊂⊢+3×⊣

To find which neighbours have different orientations, we can use our `L` function again.

In [18]:
⊢(lns lnw lne)←L¨ns nw ne
⊢(lsn lsw lse)←L¨sn sw se
⊢(lwn lws lwe)←L¨wn ws we
⊢(len les lew)←L¨en es ew

┌───────────────────────┬───────────────────────┬───────────────────────┐
│┌───────┬─────┬───────┐│┌───────┬──────┬──────┐│┌──────┬──────┬───────┐│
││l>SW-W │l>S-O│l>SE-E │││l>SW-NE│l>S-NW│l>SE-N│││l>SW-N│l>S-NE│l>SE-NW││
│├───────┼─────┼───────┤│├───────┼──────┼──────┤│├──────┼──────┼───────┤│
││l>NW-SW│l>N-S│l>NE-SE│││l>NW-E │l>N-W │l>NE-O│││l>NW-O│l>N-E │l>NE-W ││
│├───────┼─────┼───────┤│├───────┼──────┼──────┤│├──────┼──────┼───────┤│
││l>W-NW │l>O-N│l>E-NE │││l>W-SE │l>O-SW│l>E-S │││l>W-S │l>O-SE│l>E-SW ││
│└───────┴─────┴───────┘│└───────┴──────┴──────┘│└──────┴──────┴───────┘│
└───────────────────────┴───────────────────────┴───────────────────────┘

┌───────────────────────┬───────────────────────┬───────────────────────┐
│┌───────┬─────┬───────┐│┌───────┬──────┬──────┐│┌──────┬──────┬───────┐│
││l>W-SW │l>O-S│l>E-SE │││l>W-NE │l>O-NW│l>E-N │││l>W-N │l>O-NE│l>E-NW ││
│├───────┼─────┼───────┤│├───────┼──────┼──────┤│├──────┼──────┼───────┤│
││l>SW-NW│l>S-N│l>SE-NE│││l>SW-E │l>S-W │l>SE-O│││l>SW-O│l>S-E │l>SE-W ││
│├───────┼─────┼───────┤│├───────┼──────┼──────┤│├──────┼──────┼───────┤│
││l>NW-W │l>N-O│l>NE-E │││l>NW-SE│l>N-SW│l>NE-S│││l>NW-S│l>N-SE│l>NE-SW││
│└───────┴─────┴───────┘│└───────┴──────┴──────┘│└──────┴──────┴───────┘│
└───────────────────────┴───────────────────────┴───────────────────────┘

┌───────────────────────┬───────────────────────┬───────────────────────┐
│┌───────┬──────┬──────┐│┌───────┬──────┬──────┐│┌──────┬───────┬──────┐│
││l>NE-SW│l>NW-S│l>N-SE│││l>NE-W │l>NW-O│l>N-E │││l>NE-N│l>NW-NE│l>N-NW││
│├───────┼──────┼──────┤│├───────┼──────┼──────┤│├──────┼───────┼──────┤│
││l>E-NW │l>W-N │l>O-NE│││l>E-SW │l>W-S │l>O-SE│││l>E-O │l>W-E │l>O-W ││
│├───────┼──────┼──────┤│├───────┼──────┼──────┤│├──────┼───────┼──────┤│
││l>SE-W │l>SW-O│l>S-E │││l>SE-NW│l>SW-N│l>S-NE│││l>SE-S│l>SW-SE│l>S-SW││
│└───────┴──────┴──────┘│└───────┴──────┴──────┘│└──────┴───────┴──────┘│
└───────────────────────┴───────────────────────┴───────────────────────┘

┌───────────────────────┬───────────────────────┬───────────────────────┐
│┌──────┬──────┬───────┐│┌──────┬──────┬───────┐│┌──────┬───────┬──────┐│
││l>N-SW│l>NE-S│l>NW-SE│││l>N-W │l>NE-O│l>NW-E │││l>N-NE│l>NE-NW│l>NW-N││
│├──────┼──────┼───────┤│├──────┼──────┼───────┤│├──────┼───────┼──────┤│
││l>O-NW│l>E-N │l>W-NE │││l>O-SW│l>E-S │l>W-SE │││l>O-E │l>E-W │l>W-O ││
│├──────┼──────┼───────┤│├──────┼──────┼───────┤│├──────┼───────┼──────┤│
││l>S-W │l>SE-O│l>SW-E │││l>S-NW│l>SE-N│l>SW-NE│││l>S-SE│l>SE-SW│l>SW-S││
│└──────┴──────┴───────┘│└──────┴──────┴───────┘│└──────┴───────┴──────┘│
└───────────────────────┴───────────────────────┴───────────────────────┘

And the value of the constant that must multiply the change of energy is calculated as:

In [19]:
KK←{⍺{'K(1+(',⍺,'),',⍵,')'}¨⊃{'(',⍺,')+(',⍵,')'}¨/⍵}
⊢kon←lns KK lnw lne  ⍝ force factor when transforming into north cell
⊢kos←lsn KK lsw lse  ⍝ force factor when transforming into south cell
⊢kow←lwe KK lwn lws  ⍝ force factor when transforming into west cell
⊢koe←lew KK len les  ⍝ force factor when transforming into east cell

┌────────────────────────────────┬──────────────────────────────┬────────────────────────────────┐
│K(1+(l>SW-W),(l>SW-NE)+(l>SW-N))│K(1+(l>S-O),(l>S-NW)+(l>S-NE))│K(1+(l>SE-E),(l>SE-N)+(l>SE-NW))│
├────────────────────────────────┼──────────────────────────────┼────────────────────────────────┤
│K(1+(l>NW-SW),(l>NW-E)+(l>NW-O))│K(1+(l>N-S),(l>N-W)+(l>N-E)) │K(1+(l>NE-SE),(l>NE-O)+(l>NE-W))│
├────────────────────────────────┼──────────────────────────────┼────────────────────────────────┤
│K(1+(l>W-NW),(l>W-SE)+(l>W-S)) │K(1+(l>O-N),(l>O-SW)+(l>O-SE))│K(1+(l>E-NE),(l>E-S)+(l>E-SW)) │
└────────────────────────────────┴──────────────────────────────┴────────────────────────────────┘

┌────────────────────────────────┬──────────────────────────────┬────────────────────────────────┐
│K(1+(l>W-SW),(l>W-NE)+(l>W-N)) │K(1+(l>O-S),(l>O-NW)+(l>O-NE))│K(1+(l>E-SE),(l>E-N)+(l>E-NW)) │
├────────────────────────────────┼──────────────────────────────┼────────────────────────────────┤
│K(1+(l>SW-NW),(l>SW-E)+(l>SW-O))│K(1+(l>S-N),(l>S-W)+(l>S-E)) │K(1+(l>SE-NE),(l>SE-O)+(l>SE-W))│
├────────────────────────────────┼──────────────────────────────┼────────────────────────────────┤
│K(1+(l>NW-W),(l>NW-SE)+(l>NW-S))│K(1+(l>N-O),(l>N-SW)+(l>N-SE))│K(1+(l>NE-E),(l>NE-S)+(l>NE-SW))│
└────────────────────────────────┴──────────────────────────────┴────────────────────────────────┘

┌────────────────────────────────┬────────────────────────────────┬──────────────────────────────┐
│K(1+(l>NE-N),(l>NE-SW)+(l>NE-W))│K(1+(l>NW-NE),(l>NW-S)+(l>NW-O))│K(1+(l>N-NW),(l>N-SE)+(l>N-E))│
├────────────────────────────────┼────────────────────────────────┼──────────────────────────────┤
│K(1+(l>E-O),(l>E-NW)+(l>E-SW)) │K(1+(l>W-E),(l>W-N)+(l>W-S)) │K(1+(l>O-W),(l>O-NE)+(l>O-SE))│
├────────────────────────────────┼────────────────────────────────┼──────────────────────────────┤
│K(1+(l>SE-S),(l>SE-W)+(l>SE-NW))│K(1+(l>SW-SE),(l>SW-O)+(l>SW-N))│K(1+(l>S-SW),(l>S-E)+(l>S-NE))│
└────────────────────────────────┴────────────────────────────────┴──────────────────────────────┘

┌──────────────────────────────┬────────────────────────────────┬────────────────────────────────┐
│K(1+(l>N-NE),(l>N-SW)+(l>N-W))│K(1+(l>NE-NW),(l>NE-S)+(l>NE-O))│K(1+(l>NW-N),(l>NW-SE)+(l>NW-E))│
├──────────────────────────────┼────────────────────────────────┼────────────────────────────────┤
│K(1+(l>O-E),(l>O-NW)+(l>O-SW))│K(1+(l>E-W),(l>E-N)+(l>E-S)) │K(1+(l>W-O),(l>W-NE)+(l>W-SE)) │
├──────────────────────────────┼────────────────────────────────┼────────────────────────────────┤
│K(1+(l>S-SE),(l>S-W)+(l>S-NW))│K(1+(l>SE-SW),(l>SE-O)+(l>SE-N))│K(1+(l>SW-S),(l>SW-E)+(l>SW-NE))│
└──────────────────────────────┴────────────────────────────────┴────────────────────────────────┘

#### Transformation rate

Now we can find the transformation rates. The energy difference resulting from the transformation must be equal to the work that the force performs, which can be obtained multiplying the force by the velocity and time. The ratio will be the inverse of the time.

The energy difference `gon` has already been calculated. The force is obtained multiplying this energy difference by the constant `kon`, dividing by the volume and multiplying by the face area. The velocity can be obtained multiplying the boundary mobility by the pressure (`mon×gon÷V`). Therefore:

In [20]:
O rn←(⊂'A÷(V×V)') MUL mon MUL kon MUL{'(',⍵,')'}¨gon  ⍝ rate of transformation into north cell
O rs←(⊂'A÷(V×V)') MUL mos MUL kos MUL{'(',⍵,')'}¨gos  ⍝ rate of transformation into south cell
O rw←(⊂'A÷(V×V)') MUL mow MUL kow MUL{'(',⍵,')'}¨gow  ⍝ rate of transformation into west cell
O re←(⊂'A÷(V×V)') MUL moe MUL koe MUL{'(',⍵,')'}¨goe  ⍝ rate of transformation into east cell

A÷(V×V)×M(T,O-N)×K(1+(l>N-S),(l>N-W)+(l>N-E))×(G(O-N))

A÷(V×V)×M(T,O-S)×K(1+(l>S-N),(l>S-W)+(l>S-E))×(G(O-S))

A÷(V×V)×M(T,O-W)×K(1+(l>W-E),(l>W-N)+(l>W-S))×(G(O-W))

A÷(V×V)×M(T,O-E)×K(1+(l>E-W),(l>E-N)+(l>E-S))×(G(O-E))

#### High energy cells

It will be assumed that multiple points, where more than two grains meet, are high-energy regions in which the transformation will take place faster. If a cell has grain boundaries with more than one different grains, it will be considered a multiple point and we will multiply the obtained transformation rate by a factor. The value of this factor will depend on the cell being a triple or a quadruple point (for simplicity, quintuple points, where each face is a grain boundary with respect to a different grain, will also be considered quadruple points).

To find the number of grain boundaries, we will use our `l..` variables.

In [21]:
AND←{'((',⍺,')∧(',⍵,'))'}¨
O nb←'4'SUB SUM lon(lnw AND low)(lsw AND 1⊖lon)(lse AND ¯2↓¨2↓¨lne AND 1⌽low)

4 - (l>O-N+((l>N-W)∧(l>O-W))+((l>S-W)∧(l>S-O))+((l>S-E)∧(l>N-E)∧(l>E-O)))

We just need to use this number to index into a list with the corresponding factors:

In [22]:
F←{'f[3⌊(',⍵,')]'}¨
O fb←F nb

f[3⌊(4 - (l>O-N+((l>N-W)∧(l>O-W))+((l>S-W)∧(l>S-O))+((l>S-E)∧(l>N-E)∧(l>E-O))))]

The transformation rates will be obtained multiplying these factors by the values previously calculated (`fb×rn`, `fb×rs`, etc).

#### Reduction of the number of operations

Notice that there are a number of operations that we are performing several times.

To begin with, the calculation of misorientation angles is a commutative operation. If, for example, we have calculated the disorientation `N-W`, we know that the disorientation `W-N` will be the same. The same thing happens with boundary energies. So, instead of calculating `G(W-N)`, for example, we can directly use the value calculated for `G(N-W)`.

Moreover, we are repeating ourselves when we perform calculations of misorientation angles and energies at opposite faces. For example, if we have calculated the disorientations `on` (with the disorientation `O-N` in the central cell), and the corresponding boundary energies `gn` (with value `G(O-N)` at the center), the energy `gs` could be obtained simply rotating `gn` with `1⊖gn` (so that the value of the central cell will be the value previously at the south, `G(S-O)`, which is equal to `G(O-S)`).

The same reasoning applies to boundary mobilities and low angle boundaries, which depend on disorientations too.

In summary, we can significantly reduce the number of operations. These are all the magnitudes that we need to calculate and cannot be obtained just rotating other arrays:

- Disorientations: `on` `ow` `nw` `sw` `ns` `we`
- Boundary energies: `gon` `gow` `gnw` `gsw` `gns` `gwe`
- Low angle boundaries: `lon` `low` `lnw` `lsw` `lns` `lwe`
- Mobilities: `mon` `mow`

#### Non-valid points

In general, the model will use experimental data as input. In experimental EBSD measurements, it is common to find points for which the orientation measurement is not reliable. This is usually indicated by low confidence index or image quality values. Sometimes, the non valid points are assigned to a different phase.

The way to handle these points during the simulations will be to consider that they are highly deformed regions, and therefore their initial energy is high. So, we will assume that their initial boundary energy is as high as possible (`4×gh`, where `gh` is the energy at high angle grain boundaries).

We will also consider that the transformation rate into a non-valid cell is always zero.

## Iterative method

The model keeps, in addition to the current microstructure, a list of arrays with the accumulated transformation fractions for each of the neighbours. Every time step, new transformation rates are calculated and the transformation fractions are updated. When a transformation fraction reaches the value 1, the cell is transformed into that neighbour.

### Time increment

Each step, the time increment is calculated such that at least one cell is transformed. However, the user may specify a larger minimum time step. The reason is that, otherwise, the required number of steps could be too high. In practice, there will be many cells that will transform approximately at the same time. We can transform all of them in a single time step just with a longer increment, at the expense of some accuracy.

To find the minimum time increment needed to transform at least one cell, we will need to divide the remaining transformation fractions by the transformation rates and find the minimum value.

### Transformation

Now, we are ready to transform our cells. When the cell transformation fraction into one of its neighbours reaches a value of 1 (or higher), that cell will get the orientation of the neighbour. Moreover, all its transformation fractions are set to zero. This means that the transformation fraction of the cell into its neighbours becomes zero, but also the transformation fraction of the neighbours into the transformed cell.

#### Cell swapping

There will be situations during the simulation in which two neighbour cells would have to be transformed at the same time into each other. We can slightly randomise the transformation rates applying a small random perturbation, so that there will be more variation in the precise instant in which cells transform. However, this will have no effect if we also use a minimum time step parameter, particularly if this parameter is relatively large with respect to our randomization factor.

In real materials, cell swapping does not make sense. A region of material might reorient to get closer to a neighbour, but then that neighbour will not transform. Such a succese would require some kind of "jump" of the boundaries that would need much more energy than as predicted by the model.

In order to avoid cell-swapping in our simulations, we will add an additional check. Before transforming a cell, we will check if any of its neighbours is transforming into it at the same step. If this is the case, the cell is transformed only if its transformation fraction into the neighbour is higher than the transformation fraction of the neighbour.

### Output

During each time step, we will also generate some output. The evolution of average grain diameter with respect to time is written to a text file. It is also possible to write the microstructure to a file at regular time intervals.

## Model input data

The model needs an initial microstructure and a set of material and simulation parameters. All parameters are read from a [`json5`](https://json5.org/) file.

In this file, we also specify what is the input data. We can choose between solving an ideal single circular grain problem, start with a random microstructure in which each cell has a random orientation, or read data from an `ang` (EBSD) file.

Either if we are using EBSD data or we choose as initial state the circle or the random microstructure problem, we will need the microstructure array with the orientation indices, and the misorientation namespace that works with those indices.

### Circle problem

The simplest problem that can be solved by the model is a problem in which there are only two grains: a circular grain is embedded inside another grain, which fills the rest of the grid. The user specifies the radious of the grain (in cells) and the grid is defined such that there is at least one cell at every side of the circular grain. Orientations are defined such that the misorientation between both grains is as large as possible (which, for cubic symmetry, is 63.8 degrees along the `1 1 1` direction).

In [23]:
C←{Euler.(M↓⍉↑(⊃,/1(1 1 1÷3*÷2)×2 1○360÷⍨○63.8)(4⍴0))((⍵*2)>(+.×⍨)¨(⍳2⍴s)+0.5(⊣-⊣×⊢)s←2+⌈2×⍵)}

### Random microstructure

To generate ransom microstructures we will need two or three parameters. Two of them are to specify the size of the grid and the third optional parameter specifies the number of distinct orientations. If this third value is not specified, the total number of cells is used.

When we generate a random microstructure, it is not enough to randomly generate Euler angles or quaternions. What we want are uniformly distributed orientations. When using quaternons, orientations are inside the four-dimensional sphere of radious 1. What we will do is to first generate a random latitude and then apply two random rotations.

In [24]:
 R←{(x y n)←(⊢,×/)⍣(2=≢⍵)⊢⍵ ⋄ (Euler.{F←{(1 2○⊂2×○?⍺⍴0)×⊂⍵*÷2} ⋄ M UV⍵(F∘(1∘-),F)?⍵⍴0}n)(?x y⍴n)}

### EBSD (ang) files

To read and write ang files and perform other operations with EBSD data, the `EBSD` namespace is defined. Inside this namespace, we define `Read` and `Write` functions to read and write files, and also a `Crop` function to select only a region of the EBSD data. The `Orientations` function will return the unique orientations, after optionally applying some rounding, and the microstructure (the array with the orientation indices). The functions `IQ` and `CI` just return the *confidence index* and *image quality* columns from the data.

In [25]:
:Namespace EBSD                                                      ⍝ Read and write ang files

    Read←{                                                           ⍝ read ang file
        (h l)←((2-'#'=⊃¨)⊆⊢)⊃⎕NGET⍵1                                 ⍝   get (hash) comments and data
        h,⍥⊂⎕CSV⍠2⊢('(^\s+)|(\s+$)' '\s+'⎕R'' ','⊢l)'N'2 0           ⍝   format data and read as csv
    }
    Crop←{x y←⍺ ⋄ s←((x>4∘⊃)∧y>3∘⊃)⍵ ⋄ s∘/¨⍵}                        ⍝ crop to given size
    Write←{⍵⊣⍵1⎕NPUT⍨{⍺,⍵⎕CSV⍠2⍠'Separator' ' '⊢'' 'N'}/⍺}           ⍝ write comments and data to ang file
    Orientations←{                                                   ⍝ get orientations
        ⍺←0 ⋄ nx←⌊0.5+1+(⌈/x)÷⌈/|2-/x←4⊃⍵ ⋄ ny←⌊0.5+(≢x)÷nx          ⍝   find size
        g←∪ea←(a←○⍺÷180)(⌊0.5+÷⍨)⍣(⍺>0)↓⍉↑3↑⍵                        ⍝   unique euler angles
        (a×⍣(⍺>0)⊢g)(nx ny⍴g⍳ea)                                     ⍝   return angles and indices
    }
    IQ←5∘⊃ ⋄ CI←6∘⊃                                                  ⍝ image quality and confidence index
:EndNamespace

## Orientations and disorientations

Orientations are used in the model to calculate disorientations between cells. Since the microstructure is composed by grains which are usually larger than the typical cell size, with boundaries extending along several cells, it will be common to have several cells for which the same misorientations must be calculated.

Storing 3D orientations and calculating misorientations between them are relatively expensive operations, which take time and memory. For example, a quaternion is represented by four real numbers (although the use of unitary quaternions allows to optionally store only three and calculate the fourth one). Calculating a disorientation under cubic symmetry will require to calculate 24 different disorientations and choose the minimum one. That means that, in the computer, we will need to store 100 floating point numbers and perform 24 quaternion dot products, just to calculate a disorientation.

If we want to calculate millions of disorientations every step, we will need a strategy to minimize the memory and the number of calculations needed to find (often repeated) disorientations.

### Orientation index

All different orientations present in the microstructure will be stored in a list. Let's call this list `g`. The orientation of each cell is then represented as a simple integer which corresponds to the index of the orientation of the cell in the list `g`.

We will typically use EBSD data as input. It must be taken into account that the precission of EBSD experiments is limited. Orientations are calculated by the microscope software with certain error. Moreover, we want to minimise the number of unique orientations in the problem, to reduce both the memory requirements to store orientations and disorientations, and the number of calculations, and therefore the total running time.

In order to limit the number of unique orientations in the problem, the orientations, represented as triplets of Euler angles in EBSD files, will first be approximated to the closest angle with the chosen precission. Typical values will be 5 degrees for approximate (but fast) solutions, 2.5 degrees for most problems, and 1 degree to obtain higher precission results (at the expense of longer running time and memory requirements).

The function `GQ` takes as right argument an array where each element is a triplet of Euler angles (in radians). The orientation values are rounded with the precission given as left argument (in degrees), and the list of unique orientations (in radians) and indices array are returned.

In [26]:
]dinput
GQ←{
    ⍺←0                                ⍝ default: not rounding
    u←∪g←(a←○⍺÷180)(⌊0.5+÷⍨)⍣(⍺>0)⊢,⍵  ⍝ unique orientations
    (a×⍣(⍺>0)⊢u)((⍴⍵)⍴u⍳g)             ⍝ orientations and indices
}

For example, if these is our input:

In [27]:
4⍕¨(180÷○1)×ea←(○0.05)×?5 5⍴⊂3⍴0  ⍝ generate random low angles, so that there are cells with the same orientation

┌─────────────────────┬─────────────────────┬─────────────────────┬─────────────────────┬─────────────────────┐
│ 1.2865 1.9487 0.9162│ 0.2574 8.3176 7.8539│ 6.4968 6.8437 2.4338│ 4.4242 8.5030 6.9649│ 7.7328 7.5307 6.1395│
├─────────────────────┼─────────────────────┼─────────────────────┼─────────────────────┼─────────────────────┤
│ 4.4834 3.8348 6.6916│ 0.9043 6.4986 6.5530│ 4.3462 2.5162 6.6025│ 0.6894 3.8396 4.1012│ 5.2971 8.1089 7.2471│
├─────────────────────┼─────────────────────┼─────────────────────┼─────────────────────┼─────────────────────┤
│ 2.5578 1.0717 2.5556│ 7.1092 6.5157 8.8907│ 1.0298 0.0240 8.6205│ 2.7732 8.3349 3.1990│ 6.9311 5.7174 8.6618│
├─────────────────────┼─────────────────────┼─────────────────────┼─────────────────────┼─────────────────────┤
│ 3.5804 8.5652 3.8853│ 3.5442 1.6622 3.8627│ 4.6436 5.6392 5.5838│ 8.1334 5.2990 7.6661│ 7.0518 4.7357 1.5512│
├─────────────────────┼─────────────────────┼─────────────────────┼─────────────────────┼─────────────────────┤
│ 6.5554 8.9713 7.5455│ 8.3806 0.0377 5.4810│ 2.4370 2.5945 5.9824│ 3.9569 7.6951 3.2874│ 4.6646 0.5389 8.7819│
└─────────────────────┴─────────────────────┴─────────────────────┴─────────────────────┴─────────────────────┘

we will get:

In [28]:
uea q←2.5 GQ ea
(↑(180÷○1)×uea)q

┌─────────────┬──────────────┐
│2.5 2.5 0 │ 0 1 2 3 4│
│0 7.5 7.5│ 5 1 6 7 3│
│7.5 7.5 2.5│ 8 9 10 11 12│
│5 7.5 7.5│13 14 15 12 16│
│7.5 7.5 5 │17 18 14 19 20│
│5 5 7.5│ │
│5 2.5 7.5│ │
│0 5 5 │ │
│2.5 0 2.5│ │
│7.5 7.5 10 │ │
│0 0 7.5│ │
│2.5 7.5 2.5│ │
│7.5 5 7.5│ │
│2.5 7.5 5 │ │
│2.5 2.5 5 │ │
│5 5 5 │ │
│7.5 5 2.5│ │
│7.5 10 7.5│ │
│7.5 0 5 │ │
│5 7.5 2.5│ │
│5 0 10 │ │
└─────────────┴──────────────┘

### Conversion from Euler angles into quaternions

Representing orientations as Euler angles is common practice in the field of crystallography. However, Euler angles are difficult to work with when it is necessary to perform operations with the orientations. In particular, for the calculation of disorientations, it is more convenient to represent orientations as quaternions.

To find the quaternion corresponding to a triplet of Euler angles, we will first define the quaternion corresponding to each of the three rotations, and then multiply them.

First, we define `QA` to get a quaternion given the axis of rotation and angle:

In [29]:
UV←×∘(÷(÷2)*⍨+.×⍨)⍨ ⋄ QA←((⊂2○⊢),(⊂1○⊢)∘.×∘UV{⍺←z ⋄ ⍺})∘(÷∘2)

We also need a quaternion product function. We will use the [Hamilton product](https://en.wikipedia.org/wiki/Quaternion#Hamilton_product). First, we multiply the components of the left quaternion by the components of the right quaternion specified in `c`, and then multiply by the signs in `u` and add the rows to get the resulting quaternion:

In [30]:
c←↑(0 1 2 3)(1 0 3 2)(2 3 0 1)(3 2 1 0)        ⍝ product components ⎕IO=0
u←↑(1 ¯1 ¯1 ¯1)(1 1 1 ¯1)(1 ¯1 1 1)(1 1 ¯1 1)  ⍝ product unit factors
QP←+/u×⊣(×⍤1)(⊂c)⌷⊢                            ⍝ product, conjugate and dot product

Finally, we can define our `QED` function to transform Euler angles into quaternions:

In [31]:
(x y z)←=∘⊂⍨⍳3 ⋄ QE←⊃{⍺←z x z ⋄ ⍺}QP.QA⊢ ⋄ RD←180÷⍨○ ⋄ QED←QE∘RD

If we wanted to convert the orientations that we calculated before:

In [32]:
4⍕⍉↑qs←QE↓⍉↑uea

0.9995 0.0218 0.0005 0.0218
 0.9957 0.0653 ¯0.0043 0.0653
 0.9941 0.0653 0.0029 0.0870
 0.9919 0.0654 ¯0.0014 0.1086
 0.9919 0.0654 0.0014 0.1086
 0.9931 0.0436 ¯0.0010 0.1088
 0.9938 0.0218 ¯0.0005 0.1088
 0.9981 0.0436 ¯0.0019 0.0436
 0.9990 0.0000 0.0000 0.0436
 0.9862 0.0654 ¯0.0014 0.1518
 0.9979 0.0000 0.0000 0.0654
 0.9969 0.0654 0.0000 0.0435
 0.9905 0.0436 0.0000 0.1304
 0.9957 0.0654 ¯0.0014 0.0653
 0.9976 0.0218 ¯0.0005 0.0654
 0.9952 0.0436 0.0000 0.0871
 0.9952 0.0436 0.0019 0.0871
 0.9877 0.0872 0.0000 0.1300
 0.9941 0.0000 0.0000 0.1089
 0.9957 0.0654 0.0014 0.0653
 0.9914 0.0000 0.0000 0.1305

### Misorientation level

A misorientation can be calculated as a quaternion that represents the rotation between both orientations. We will only be interested in the misorientation angle, also called *disorientation*. Of course, a disorientation is always in the range between 0 and 360 degrees. Under cubic symmetry, it can be proven that the maximum disorientation is 62.8 degrees.

It can be specified, as a user parameter, what precission will be used for misorientation angles. A typicial value is 0.5 degrees. Then, it will be possible to represent the index with a single byte (in fact, it would be enough with 7 bits).

The `ML` function takes a misorientation angle in radians as right argument and an optional angle step in degrees as left argument.

In [33]:
ML←{⍺←0.5 ⋄ ⌊(180×⍵)÷○⍺}

### Calculation of misorientation angle

We can easily calculate the quaternion necessary to rotate a quaternion into another. But, since we are only interested in the misorientation angle, we only need to calculate the scalar part of the product. We can obtain this scalar part as the dot product by the conjugate. However, we must also take symmetry into account, testing all the possible variants and chosing the minimum angle.

#### Dot product

The dot product of two quaternions (obtained, like for vectors, multiplying and adding their elements), after getting the conjugate of one of them, will result in the cosine of the half-angle between both quaternions.

In [34]:
QC←×∘(1,-3⍴1) ⋄ QD←+.×  ⍝ product, conjugate and dot product

#### Cubic symmetry

There are a total of 24 variants. Each of these variants can be represented as a rotation:

In [35]:
QAD←QA∘RD
cs ← ⊂1 0 0 0                                                ⍝   identity
cs,←,(1 0 0)(0 1 0)(0 0 1)∘.QAD 90 180 270                   ⍝   4-fold around <001>
cs,←,(1 1 1)(¯1 1 1)(1 ¯1 1)(1 1 ¯1)∘.QAD 120 240            ⍝   3-fold around <111>
cs,←,(1 1 0)(1 0 1)(0 1 1)(1 ¯1 0)(¯1 0 1)(0 1 ¯1)∘.QAD 180  ⍝   2-fold around <110>

When we have an orientation represented by a quaternion, we can compose it with one of these rotations, multiplying the quaternions, to obtain the symmetric variants of our orientation.

#### Disorientation under symmetry

To calculate disorientations between two orientations, represented as two quaternions, under symmetry, we will first multiply one of the quaternions by the conjugate of the other, and then will calculate the 24 dot products of the symmetry quaternions with the conjugate of the result. Finally, we choose the maximum value, which will be the maximum cosine of the half angle, and calculate the corresponding angle.

In [36]:
_D←{⊃2×¯2○1⌊⍺⍺⌈.(|QD)⊂QC⍺QP QC⍵} ⋄ DC←cs _D  ⍝ disorientation under symmetry ⍺⍺ and under cubic symmetry

### Memoization

As previously said, many boundaries will be equal, between the same pair of orientations. Since calculating disorientations is an expensive operation, it will be convenient to avoid calculating the same one twice. The technique used to achieve this goal is called *memoization* and consists in storing the results calculated for each pair of arguments. If the same result is needed again, the stored result is retrieved and returned, instead of being calculated again.

Taking into account that calculating the disorientation for a pair of orientations is a commutative operation, and that the disorientation of an orientation with itself will always be zero (and there is no need to store all those zeros), we will need a total of `n×(n-1)÷2` elements for `n` orientations. We will initialize all disorientations to a negative value, so that we know that it is not a valid misorientation level. The function `MC` returns a initiallised misorientations cache of the right size for `⍵` orientations.

In [37]:
MC←{¯1⍴⍵×(⍵-1)÷2}

The function `IM` returns the index of the misorientation given two pairs of orientation indices. It assumes that `⍺` is larger than `⍵`. If we want it to work with any `⍺≠⍵` we can run instead the train `⌈IM⌊`.

In [38]:
IM←{⍵+⍺×(⍺-1)÷2}

Finally, the function `M` will return a namespace `m` that contains an `m.M` function which can be used to get the misorientation angle between two orientations (or two sets of orientations). This function will check if both orientations are the same and return 0, or will try to retrieve a previously calculated value from the cache. The disorientations that are not stored in the cache, will be calculated and stored before being returned.

In order to define `m.M`, we first define an operator `_M_` which takes as operands the orientations to consider and the cache of misorientations (returned by `MC`). In addition to the `M` function, the `m` namespace includes the functions `MC` and `IM`, a curried version of `ML`, as well as a degrees version (simply called `L`). The `D` function performs the inverse of `L`.

In [39]:
]dinput
M←{⍺←0.5                                                         ⍝ namespace to calculate misorientations
    IM←⌈(⊢+2÷⍨⊣×1-⍨⊣)⌊ ⋄ CM←{(c←⎕NS⍬).m←¯1⍴⍨1+⍵IM.-1 2 ⋄ c}      ⍝ index and cache of misorientations
    _M_ ← {(c s)←⍺⍺ ⍵⍵                                           ⍝ memoization
        (a w)←(⊂,d←⍺≠⍵)/∘,¨⍺ ⍵ ⋄ 0=≢a:d                          ⍝   return zeros if no different pairs
        n←(≠,i)∧s.m[i←a IM w]<0 ⋄ 0=≢p←n/⍥,i:s.m[i]@⊢d           ⍝   return calculated if all done
        s.m[p]←ML⊃a MC⍥{↓⍉↑c[n/⍵]}w ⋄ s.m[i]@⊢d                  ⍝   calculate and return
    }
    m←⎕NS'MC' 'IM' ⋄ m.ML←⍺∘ML ⋄ m.L←m.ML RD                     ⍝ namespace with curried ML
    m.D←(⍺÷2)+⍺∘× ⋄ w←↓⍉↑⍵ ⋄ m.M←w _M_(CM≢w) ⋄ m                 ⍝ degrees and misorientation functions
}

### Euler namespace

All orientations and misorientations related operations are defined in the `Euler` namespace.

In [40]:
:Namespace Euler                                                     ⍝ Orientations and misorientations

    c←↑(0 1 2 3)(1 0 3 2)(2 3 0 1)(3 2 1 0)                          ⍝ product components ⎕IO=0
    u←↑(1 ¯1 ¯1 ¯1)(1 1 1 ¯1)(1 ¯1 1 1)(1 1 ¯1 1)                    ⍝ product unit factors
    QP←+/u×⊣(×⍤1)(⊂c)⌷⊢ ⋄ QC←×∘(1,-3⍴1) ⋄ QD←+.×∘QC                  ⍝ product, conjugate and dot product
                                                                     ⍝ quaternion from Euler angles
    RD←180÷⍨○ ⋄ UV←×∘(÷(÷2)*⍨+.×⍨)⍨ ⋄ (x y z)←=∘⊂⍨⍳3                 ⍝   degrees, unitary vector and unit vectors
    QA←((⊂2○⊢),(⊂1○⊢)∘.×∘UV{⍺←z ⋄ ⍺})∘(÷∘2) ⋄ QAD←QA∘RD              ⍝   quaternion from axis-angle
    QE←⊃{⍺←z x z ⋄ ⍺}QP.QA⊢ ⋄ QED←QE∘RD                              ⍝   quaternion from Euler angles (zxz)
                                                                     ⍝ cubic symmetry
    cs ← ⊂1 0 0 0                                                    ⍝   identity
    cs,←,(1 0 0)(0 1 0)(0 0 1)∘.QAD 90 180 270                       ⍝   4-fold around <001>
    cs,←,(1 1 1)(¯1 1 1)(1 ¯1 1)(1 1 ¯1)∘.QAD 120 240                ⍝   3-fold around <111>
    cs,←,(1 1 0)(1 0 1)(0 1 1)(1 ¯1 0)(¯1 0 1)(0 1 ¯1)∘.QAD 180      ⍝   2-fold around <110>
    MC←⊃2×¯2○1⌊cs⌈.(|QD)∘⊂QP∘QC ⋄ ML←⌊○⍤⊣÷⍨180×⊢                     ⍝ misorientation (cubic symmetry) and level
    M←{⍺←0.5                                                         ⍝ namespace to calculate misorientations
        IM←⌈(⊢+2÷⍨⊣×1-⍨⊣)⌊ ⋄ CM←{(c←⎕NS⍬).m←¯1⍴⍨1+⍵IM.-1 2 ⋄ c}      ⍝ index and cache of misorientations
        _M_ ← {(c s)←⍺⍺ ⍵⍵                                           ⍝ memoization
            (a w)←(⊂,d←⍺≠⍵)/∘,¨⍺ ⍵ ⋄ 0=≢a:d                          ⍝   return zeros if no different pairs
            n←(≠,i)∧s.m[i←a IM w]<0 ⋄ 0=≢p←n/⍥,i:s.m[i]@⊢d           ⍝   return calculated if all done
            s.m[p]←ML⊃a MC⍥{↓⍉↑c[n/⍵]}w ⋄ s.m[i]@⊢d                  ⍝   calculate and return
        }
        m←⎕NS'MC' 'IM' ⋄ m.ML←⍺∘ML ⋄ m.L←m.ML RD                     ⍝ namespace with curried ML
        m.D←(⍺÷2)+⍺∘× ⋄ w←↓⍉↑⍵ ⋄ m.M←w _M_(CM≢w) ⋄ m                 ⍝ degrees and misorientation functions
    }
:EndNamespace

## Disorientation dependent magnitudes

There is a number of magnitudes in the model which depend on the value of the misorientation angle between pairs of grains.

Since we will use a discrete number of misorientation levels (as defined by the `ML` function), we will only need to calculate said magnitudes at these discrete values. This can be done only once at the beginning of the simulation. And then, during the iterative process, we will only need to get the previously calculated values.

### Boundary energy

It is obtained according to the Read-Shockley equation:

$$G_B(\theta_{ij}) = G_0 \frac{\theta_{ij}}{\theta_{HAGB}}\left(1 - \log\left(\frac{\theta_{ij}}{\theta_{HAGB}}\right)\right)$$

where $\theta_{ij}$ is the misorientation angle between cells $i$ and $j$, $\theta_{HAGB}$ is the high angle grain boundary, and $G_0$ is a material constant (eg: 3000 J/mol). In APL:

In [41]:
G0←3000 ⋄ hagb←15
g←(G0×⊢×1-⍟)@(0∘<)¨1⌊h÷⍨⍳1+h←ML(180÷⍨○)hagb

In [42]:
]plot g

Created by Causeway SVG engine - SharpPlot v3.71.0 
 
 Paint the paper ===== 
 
 
 
 Border ===== 
 Region ===== 
 X-Axis Ticks ===== 
 X-Axis tickmarks 
 
 Y-Axis Ticks ===== 
 Y-Axis tickmarks 
 
 Axes ===== 
 
 Y-axis labels 
 
 0 
 500 
 1000 
 1500 
 2000 
 2500 
 3000 
 
 for X-axis labels 
 
 0 
 2 
 4 
 6 
 8 
 10 
 12 
 14 
 16 
 18 
 20 
 22 
 24 
 26 
 28 
 30 
 32 
 
 Heading, subheading and footnotes ===== 
 Start of Line Chart =========== 
 Points follow ... 
 Line 
 
 
 
 Reset to original origin

### Boundary mobility

Boundary mobilities are calculated using Humphrey's law:

$$M(T, \theta_{ij})=M_0 \exp\left(\frac{-Q_g}{8.314\,T}\right) \left(1-\exp\left(-A\left(\frac{\theta_{ij}}{\theta_{HAGB}}\right)^n\right)\right)$$

where $Q_g$ is the activation energy for boundary migration and $T$ is the temperature. $M_0$, $A$ and $n$ are material constants.

In [43]:
M0←2.84E¯6 ⋄ Qg←140E3 ⋄ A←5 ⋄ n←4
T←1273 ⋄ m←(M0×*-Qg÷8.314×T)×1-*-A×n*⍨h÷⍨⍳1+h  ⍝ grain boundary mobility at 1000 C

In [44]:
]plot m

Created by Causeway SVG engine - SharpPlot v3.71.0 
 
 Paint the paper ===== 
 
 
 
 Border ===== 
 Region ===== 
 X-Axis Ticks ===== 
 X-Axis tickmarks 
 
 Y-Axis Ticks ===== 
 Y-Axis tickmarks 
 
 Axes ===== 
 
 Y-axis labels 
 
 0 
 5E-13 
 1E-12 
 1.5E-12 
 2E-12 
 2.5E-12 
 3E-12 
 3.5E-12 
 4E-12 
 4.5E-12 
 5E-12 
 5.5E-12 
 
 for X-axis labels 
 
 0 
 2 
 4 
 6 
 8 
 10 
 12 
 14 
 16 
 18 
 20 
 22 
 24 
 26 
 28 
 30 
 32 
 
 Heading, subheading and footnotes ===== 
 Start of Line Chart =========== 
 Points follow ... 
 Line 
 
 
 
 Reset to original origin